In [15]:
from datasets import AlphaposeSkeletonData
from train import train, _A
from model import GCN
import torch
import numpy as np


In [16]:
train_json_dir = "C:\\Users\\user\\Desktop\\alphapose_json"
train_image_dir = "C:\\Users\\user\\Desktop\\RE-ID\\datasets\\DukeMTMC-reID\\query"
valid_json_dir = "C:\\Users\\user\\Desktop\\alphapose_json_valid"
valid_image_idr =  "C:\\Users\\user\\Desktop\\RE-ID\\datasets\\DukeMTMC-reID\\query"

train_dataset = AlphaposeSkeletonData(train_json_dir, train_image_dir)
valid_dataset = AlphaposeSkeletonData(valid_json_dir, valid_image_idr)
train_dataset.__len__(), valid_dataset.__len__()

(636, 128)

In [17]:
model = GCN(3, 4, torch.tensor(_A).float())
model_name = "GCN"
train(model, train_dataset, valid_dataset, model_name = model_name, epochs= 50000, batch_size=32, learning_rate = 1e-3, early_stop_cnt=10)

learning_rate : 0.001
batch_size : 32
epochs : 50000
Epoch 1
---------------------------------
loss: 1.432277 [    0/  636]
loss: 1.183313 [  100/  636]
loss: 1.328662 [  200/  636]
loss: 1.617154 [  300/  636]
loss: 1.234867 [  400/  636]
loss: 1.011583 [  500/  636]
loss: 1.245735 [  600/  636]
val_loss: 14.228757
Save best model : 14.22875690460205 > 14.22875690460205
---------------------------------
Epoch 2
---------------------------------
loss: 1.258164 [    0/  636]
loss: 1.488250 [  100/  636]
loss: 0.858259 [  200/  636]
loss: 1.526513 [  300/  636]
loss: 1.961675 [  400/  636]
loss: 1.462455 [  500/  636]
loss: 0.859269 [  600/  636]
val_loss: 12.705671
Save best model : 12.705671310424805 > 12.705671310424805
---------------------------------
Epoch 3
---------------------------------
loss: 0.965873 [    0/  636]
loss: 1.142391 [  100/  636]
loss: 1.170014 [  200/  636]
loss: 0.924680 [  300/  636]
loss: 1.100514 [  400/  636]
loss: 0.980398 [  500/  636]
loss: 1.021392 [  6

In [18]:
_target = np.array(['front', 'back', 'left', 'right'])
model = GCN(3, 4, torch.tensor(_A).float())

model.load_state_dict(torch.load("exp/GCN/BestModel.pth"))

model.eval()
s = 0
with torch.no_grad():
    for X, y in valid_dataset:
        pred = model(X.unsqueeze(0))
        s += int(_target[pred.argmax()]== _target[y.argmax()])

print("accruacy : ", s/valid_dataset.__len__())


accruacy :  0.90625


In [19]:
from classificatoin import image_classification
json_file = "C:\\Users\\user\\Desktop\\AlphaPose\\examples\\all\\alphapose-results1.json"
image_dir = "C:\\Users\\user\\Desktop\\RE-ID\\datasets\\DukeMTMC-reID\\query"
save_dir = "C:\\Users\\user\\Desktop\\experiemnt"
image_classification(model,json_file,image_dir,save_dir)

left
0005_c2_f0046985.jpg
left
0005_c5_f0051781.jpg
left
0005_c6_f0030883.jpg
left
0019_c1_f0051470.jpg
left
0019_c2_f0055712.jpg
left
0019_c5_f0061106.jpg
right
0019_c6_f0040430.jpg
right
0021_c1_f0052336.jpg
back
0021_c6_f0030737.jpg
left
0021_c7_f0035375.jpg
left
0021_c8_f0008756.jpg
left
0023_c1_f0052564.jpg
left
0023_c2_f0056463.jpg
left
0023_c3_f0033770.jpg
back
0023_c4_f0031504.jpg
back
0025_c1_f0053650.jpg
right
0025_c8_f0010986.jpg
right
0027_c1_f0054309.jpg
right
0027_c2_f0054638.jpg
right
0027_c3_f0030042.jpg
front
0027_c4_f0023987.jpg
back
0030_c1_f0057043.jpg
right
0030_c2_f0056822.jpg
right
0030_c3_f0031952.jpg
front
0030_c4_f0026288.jpg
left
0031_c1_f0057180.jpg
left
0031_c2_f0062046.jpg
left
0031_c5_f0066705.jpg
right
0031_c6_f0045932.jpg
left
0033_c1_f0057706.jpg
left
0033_c2_f0061704.jpg
left
0033_c5_f0066543.jpg
front
0033_c6_f0045755.jpg
back
0034_c1_f0057636.jpg
front
0034_c2_f0057453.jpg
back
0035_c1_f0057672.jpg
right
0035_c2_f0057892.jpg
front
0039_c1_f0058725.j

In [1]:
from datasets import AlphaposeSkeletonData
from train import train, _A
from model import GCN
import torch
import numpy as np
train_json_dir = "C:\\Users\\user\\Desktop\\alphapose_json"
train_image_dir = "C:\\Users\\user\\Desktop\\RE-ID\\datasets\\DukeMTMC-reID\\query"
valid_json_dir = "C:\\Users\\user\\Desktop\\alphapose_json_valid"
valid_image_idr =  "C:\\Users\\user\\Desktop\\RE-ID\\datasets\\DukeMTMC-reID\\query"

train_dataset = AlphaposeSkeletonData(train_json_dir, train_image_dir)
valid_dataset = AlphaposeSkeletonData(valid_json_dir, valid_image_idr)
train_dataset.__len__(), valid_dataset.__len__()
_target = np.array(['front', 'back', 'left', 'right'])
model = GCN(3, 4, torch.tensor(_A).float())

model.load_state_dict(torch.load("exp/GCN/loss_5.129523754119873.pth"))

model.eval()
s = 0
with torch.no_grad():
    for X, y in valid_dataset:
        pred = model(X.unsqueeze(0))
        s += int(_target[pred.argmax()]== _target[y.argmax()])

print("accruacy : ", s/valid_dataset.__len__())
from classificatoin import image_classification
json_file = "C:\\Users\\user\\Desktop\\AlphaPose\\examples\\all\\alphapose-results1.json"
image_dir = "C:\\Users\\user\\Desktop\\RE-ID\\datasets\\DukeMTMC-reID\\query"
save_dir = "C:\\Users\\user\\Desktop\\experiemnt"
datasets = image_classification(model,json_file,image_dir,save_dir)

accruacy :  0.8629032258064516
left
0005_c2_f0046985.jpg
left
0005_c5_f0051781.jpg
left
0005_c6_f0030883.jpg
left
0019_c1_f0051470.jpg
left
0019_c2_f0055712.jpg
left
0019_c5_f0061106.jpg
right
0019_c6_f0040430.jpg
right
0021_c1_f0052336.jpg
back
0021_c6_f0030737.jpg
back
0021_c7_f0035375.jpg
left
0021_c8_f0008756.jpg
left
0023_c1_f0052564.jpg
left
0023_c2_f0056463.jpg
left
0023_c3_f0033770.jpg
back
0023_c4_f0031504.jpg
back
0025_c1_f0053650.jpg
right
0025_c8_f0010986.jpg
right
0027_c1_f0054309.jpg
right
0027_c2_f0054638.jpg
right
0027_c3_f0030042.jpg
front
0027_c4_f0023987.jpg
back
0030_c1_f0057043.jpg
right
0030_c2_f0056822.jpg
right
0030_c3_f0031952.jpg
front
0030_c4_f0026288.jpg
left
0031_c1_f0057180.jpg
left
0031_c2_f0062046.jpg
left
0031_c5_f0066705.jpg
front
0031_c6_f0045932.jpg
left
0033_c1_f0057706.jpg
left
0033_c2_f0061704.jpg
left
0033_c5_f0066543.jpg
front
0033_c6_f0045755.jpg
back
0034_c1_f0057636.jpg
front
0034_c2_f0057453.jpg
back
0035_c1_f0057672.jpg
right
0035_c2_f00578

In [5]:
len(datasets.dic.keys())

2116

In [6]:
for a, b in datasets:
    print(a, b)

In [8]:
next(iter(datasets.__getitem__))

TypeError: 'method' object is not iterable

In [15]:
list(datasets.dic.keys())

['0005_c2_f0046985.jpg',
 '0005_c5_f0051781.jpg',
 '0005_c6_f0030883.jpg',
 '0019_c1_f0051470.jpg',
 '0019_c2_f0055712.jpg',
 '0019_c5_f0061106.jpg',
 '0019_c6_f0040430.jpg',
 '0021_c1_f0052336.jpg',
 '0021_c6_f0030737.jpg',
 '0021_c7_f0035375.jpg',
 '0021_c8_f0008756.jpg',
 '0023_c1_f0052564.jpg',
 '0023_c2_f0056463.jpg',
 '0023_c3_f0033770.jpg',
 '0023_c4_f0031504.jpg',
 '0025_c1_f0053650.jpg',
 '0025_c8_f0010986.jpg',
 '0027_c1_f0054309.jpg',
 '0027_c2_f0054638.jpg',
 '0027_c3_f0030042.jpg',
 '0027_c4_f0023987.jpg',
 '0030_c1_f0057043.jpg',
 '0030_c2_f0056822.jpg',
 '0030_c3_f0031952.jpg',
 '0030_c4_f0026288.jpg',
 '0031_c1_f0057180.jpg',
 '0031_c2_f0062046.jpg',
 '0031_c5_f0066705.jpg',
 '0031_c6_f0045932.jpg',
 '0033_c1_f0057706.jpg',
 '0033_c2_f0061704.jpg',
 '0033_c5_f0066543.jpg',
 '0033_c6_f0045755.jpg',
 '0034_c1_f0057636.jpg',
 '0034_c2_f0057453.jpg',
 '0035_c1_f0057672.jpg',
 '0035_c2_f0057892.jpg',
 '0039_c1_f0058725.jpg',
 '0039_c2_f0063073.jpg',
 '0039_c3_f0040399.jpg',
